In [17]:
import threading # Импортируем модуль threading для запуска в отдельном потоке
import tensorflow
import keras
from flask import Flask # Импортируем модуль Flask 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from PIL import Image
import PIL.ImageOps  

In [18]:
from flask import Flask, request # request - это объекты, которыми управляет flask handles (get, set, post и т.д.)
from flask import render_template # этот объект автоматически зарендерит нам шаблон html
import imageio # для работы с изображениями
import numpy as np
import tensorflow.keras.models
import re # регулярные питоновские выражения
from tensorflow.keras.models import model_from_json
import sys 
import os
from tensorflow.keras.preprocessing import image
# Задаем имя серверу
app = Flask(__name__)

import tensorflow as tf
import base64 # универсальный формат кодирования (часто используется с изображениями)

In [19]:
# Открываем json файл разметки модели
json_file = open('model.json','r') 
loaded_model_json = json_file.read() # считываем
json_file.close() # закрываем
loaded_model = model_from_json(loaded_model_json) # используем керас, чтобы считать разметку архитектуры

In [20]:
loaded_model.load_weights("model.h5") # подгружаем веса
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
# декодируем изображение из сырого(бинарного) представления в его base64 
def convertImage(imgData1):
    imgstr = re.search(b'base64,(.*)',imgData1).group(1) # находим строчку, в которой содержится побитовое представление изображение,
                                                        # возвращает вторую(1) подгруппу
    with open('output.png','wb') as output: # временный файл, в который записываем декодированное представление
        output.write(base64.b64decode(imgstr))

In [22]:
# # Указываем адрес по которому будет откликаться Web сервис и задаем саму функцию веб сервиса. Это наша стартовая страница
@app.route('/')
@app.route('/index')
def index():
    # рендерим заранее построенный HTML файл
    return render_template("index.html")

In [23]:
# маршрут, по которому будет идти предсказание
@app.route('/predict/',methods=['GET','POST'])
def predict():
    # в любой момент, когда пользователь вызовет predict
    # после того, как нарисовал картинку, произойдет
    # inference/внедрение, вернется результат классификации
    
    # получить сырой формат изображения (этот метод объекта Flask)
    imgData = request.get_data()
    
    # кодируем в подходящий формат
    convertImage(imgData)
    print ("debug")
    # считываем изображение в память
    img = image.load_img('output.png', target_size=(28,28), color_mode = 'grayscale')
    img = PIL.ImageOps.invert(img)
    sample = np.array(img)
    sample = np.reshape(sample, (1, 28, 28, 1))
    sample = sample.astype('float32') 
    sample = sample / 255
    out = loaded_model.predict(sample)
    print ("debug2")               
    print(out)
    print(np.argmax(out,axis=1))
    print ("debug3")

# преобразуем ответ в строку
    response = np.array_str(np.argmax(out,axis=1))
    return response
        # обнуляем граф, чтобы можно было ввести новые данные
    #tf.reset_default_graph()

In [ ]:
# #     # этот порт выделяем под прогонку
port = int(os.environ.get('PORT', 5002))
    
    # прогоняем на локальном сервере по заданному порту
app.run(host='127.0.0.1', port=port, threaded=False)    
    # опционально, если хотите прогнать в режиме дебага
    #app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Dec/2021 20:14:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Dec/2021 20:14:36] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[1.3048752e-06 1.8881185e-06 6.1775672e-06 1.9167624e-02 9.1960637e-06
  8.9785463e-01 2.3653984e-06 2.0912257e-05 7.6766373e-03 7.5259261e-02]]
[5]
debug3


127.0.0.1 - - [14/Dec/2021 20:14:37] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[1.4278004e-14 6.8501770e-08 1.4960045e-02 9.8503762e-01 2.1645043e-13
  1.8022044e-11 9.0487784e-14 1.9264187e-06 3.0438886e-07 6.5802558e-11]]
[3]
debug3


127.0.0.1 - - [14/Dec/2021 20:14:40] "POST /predict/ HTTP/1.1" 200 -


debug
debug2
[[5.3877337e-04 2.5621073e-05 1.5268223e-02 9.6920413e-01 2.8169072e-07
  1.1995634e-03 3.7354668e-05 7.7691379e-07 1.3513190e-02 2.1203022e-04]]
[3]
debug3
